In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
from torch.utils.data import DataLoader
from PIL import Image
from dataloader import Drone
from utils import dataset_split
from torchvision import transforms
from model import Segmentor
from utils import calculate_iou
from engine import train_val

Splitting dataset into train/test/val --> 280/60/60

In [ ]:
parent_dir = "../Datasets/Aerial-Semantic-Segmentation-Drone-Dataset"
img_dir = "../Datasets/Aerial-Semantic-Segmentation-Drone-Dataset/dataset/semantic_drone_dataset/original_images"
masks_dir = "../Datasets/Aerial-Semantic-Segmentation-Drone-Dataset/RGB_color_image_masks/RGB_color_image_masks"

In [ ]:
images , masks = dataset_split(img_dir , masks_dir)
train , test , val = images[0] , images[1] , images[2]
train_mask , test_mask , val_mask = masks[0] , masks[1] , masks[2]

Checking the effect of downsizing the images

In [ ]:
# Open the original image
image = Image.open(os.path.join(img_dir, train[0])).convert("RGB")

trans = transforms.Compose([
    transforms.Resize((512, 512))  # Use a smaller size to see the downsizing effect
])

# Apply the transformation
image_2 = trans(image)

image = np.array(image)
image_2 = np.array(image_2)

# Plot the original image
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title("Original Image (4000 , 6000)")
plt.show()

# Plot the transformed (resized) image
plt.figure(figsize=(8, 8))
plt.imshow(image_2)
plt.title("Resized Image (512 x 512)")
plt.show()

In [ ]:
#HYPERPARAMETERS
BATCH_SIZE = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
scheduler = "exponential"
epochs = 10

In [ ]:
train_dataset = Drone(parent_dir=parent_dir , 
                      images = train , 
                      masks = train_mask)

val_dataset = Drone(parent_dir=parent_dir , 
                      images = val , 
                      masks = val_mask)

test_dataset = Drone(parent_dir=parent_dir , 
                      images = test , 
                      masks = test_mask)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
len(train_loader) , len(test_loader) , len(val_loader)

In [ ]:
train_iter = iter(train_loader)
batch = next(train_iter)
images , masks = batch

In [ ]:
index = 0 # sample number from the batch
mask = masks[index].squeeze()
image = images[index]
image.shape

In [ ]:
mask = mask.unsqueeze(dim = 0).permute(1 , 2 , 0)
mask = mask.numpy()
img = image.permute(1 , 2 , 0)
img = img.numpy()

fig , axs = plt.subplots(nrows = 1 , ncols = 2 , figsize = (10 , 10))
axs[0].imshow(img)
axs[0].set_title("Original Image")
axs[0].axis('off')
axs[1].imshow(mask)
axs[1].set_title("Mask")
axs[1].axis('off')
plt.show()


In [ ]:
model = Segmentor().to(device)
summary(model , input_size=( 3 , 512 , 512))

Training

In [ ]:
train_loss , train_iou , val_loss , val_iou = train_val(model = model , 
                                                        train_loader = train_loader, 
                                                        val_loader = val_loader, 
                                                        epochs = epochs, 
                                                        lr = lr, 
                                                        device = device , 
                                                        lr_schedule=scheduler , 
                                                        out_dir = "weights/best_checkpoint.pth" , 
                                                        neptune_config="../neptune.json")